In [1]:
import bittensor as bt
from substrateinterface import SubstrateInterface

substrate = SubstrateInterface(
    url="wss://archive.chain.opentensor.ai:443/",
    ss58_format=42,
    type_registry_preset='legacy'
)

subtensor = bt.subtensor(network="wss://archive.chain.opentensor.ai:443")

current_block = substrate.get_block()['header']['number']

block_hash = substrate.get_block_hash(current_block)

network_immunity_period = substrate.query(
        "SubtensorModule", 
        "NetworkImmunityPeriod", 
        block_hash=block_hash
    )


def query_storage(name, block):
    block_hash = substrate.get_block_hash(block)
    liquid_alpha_on = []
    for netuid in range(45):
        result = substrate.query(
            "SubtensorModule", 
            name, 
            params = [netuid],
            block_hash=block_hash
        )
        liquid_alpha_on.append(result.value)
    return liquid_alpha_on

network_registered_block = query_storage("NetworkRegisteredAt", current_block)

for netuid, block in enumerate(network_registered_block):
    if current_block - block < network_immunity_period:
        print(f"SN {netuid} in immunity period")

SN 15 in immunity period
SN 38 in immunity period


In [24]:
# commit_reveal_on = query_storage("CommitRevealWeightsEnabled", current_block)
# liquid_alpha_on = query_storage("LiquidAlphaOn", current_block)
last_adjustment_block = query_storage("LastAdjustmentBlock", current_block)
last_adjustment_block_2 = query_storage("LastAdjustmentBlock", current_block - 360)
block_since_last_step = query_storage("BlocksSinceLastStep", current_block)
tempo = query_storage("Tempo", current_block)